In [2]:
import time
import uuid
import boto3
import sagemaker

sess = sagemaker.Session()
region = boto3.Session().region_name
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

In [3]:
model_training_s3_uri = 's3://dsoaws/nyc-taxi-orig-cleaned-dropped-parquet-all-years-multiple-files-100GB/'

In [4]:
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost


instance_type = "ml.m5.24xlarge"
output_content_type = "parquet"

train_content_type = (
    "application/x-parquet" if output_content_type.upper() == "PARQUET"
    else "text/csv"
)
train_input = sagemaker.inputs.TrainingInput(
    s3_data=model_training_s3_uri,
    content_type=train_content_type,
    distribution='ShardedByS3Key',
    input_mode='FastFile'
)


In [5]:
hyperparameters = {
    "eta": "0.2",
    "gamma": "4",
    "max_depth": "5",
    "min_child_weight": "6",
    "num_round": "50",
    "objective": "reg:squarederror",
    "subsample": "0.7",
#    "verbosity": "2",
    "content_type":"parquet",
}


In [6]:
from sagemaker.pytorch import PyTorch

# PyTorch Image is used to enable distributed training
estimator = PyTorch(
    source_dir="code",
    entry_point="train_xgboost_nyctaxi.py",
    role=role,
    subnets=None,
    security_group_ids=None,
    instance_count=2, 
    instance_type="ml.m5.24xlarge",
    framework_version="1.13",
    hyperparameters=hyperparameters,
    py_version="py39"
)
estimator.fit({"train": train_input})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-03-13-04-30-31-086


2023-03-13 04:30:31 Starting - Starting the training job...
2023-03-13 04:30:48 Starting - Preparing the instances for training......
2023-03-13 04:32:02 Downloading - Downloading input data
2023-03-13 04:32:02 Training - Downloading the training image.........
2023-03-13 04:33:03 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-03-13 04:33:33,070 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-03-13 04:33:33,071 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-03-13 04:33:33,073 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-03-13 04:33:33,083 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-03-13 04:33:33,085 sagemaker_pytorch_container.training INFO     Invoking user traini

UnexpectedStatusException: Error for Training job pytorch-training-2023-03-13-04-30-31-086: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "NameError: name 'get_dmatrix' is not defined"
Command "/opt/conda/bin/python3.9 train_xgboost_nyctaxi.py --content_type parquet --eta 0.2 --gamma 4 --max_depth 5 --min_child_weight 6 --num_round 50 --objective reg:squarederror --subsample 0.7", exit code: 1